In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf

C:\Users\Shaab\repos\AI DEL 2\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


# 1. Förbereder Datan

##### 1.1 Laddar datasetet

In [2]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

C:\Users\Shaab\repos\AI DEL 2\.venv\Lib\site-packages\keras\src\datasets\cifar.py:18: VisibleDeprecationWarning: dtype(): align should be passed as Python or NumPy boolean but got `align=0`. Did you mean to pass a tuple to create a subarray type? (Deprecated NumPy 2.4)
  d = cPickle.load(f, encoding="bytes")


##### 1.2 Delar upp data mellan train och test

In [3]:
# För att testa koden snabbt, begränsa till 5000 bilder först.
# När allt funkar, kommentera bort dessa två rader för att köra "på riktigt".
x_train = x_train[:5000]
y_train = y_train[:5000]
x_test = x_test[:1000]  # Vi minskar testdatan också för snabbhetens skull
y_test = y_test[:1000]

##### 1.3 Normalisera och One-hot koda

In [4]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
y_train = to_categorical(y_train, 100)
y_test = to_categorical(y_test, 100)

# 2. Bygga Modellen (Arkitekturen)

##### 2.1 : Skapa en funktion som bygger modellen

In [5]:
def build_model(hp):
    model = keras.Sequential()

    # Conv Block 1
    # hp.Int låter tunern testa olika värden, t.ex. 32, 64 eller 96 filter
    model.add(layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=96, step=32),
        kernel_size=(3, 3),
        activation='relu',
        padding='same',
        input_shape=(32, 32, 3)
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    

    # Här testar vi olika Dropout-värden (t.ex. 0.2, 0.3, 0.4)
    model.add(layers.Dropout(hp.Float('dropout_1', min_value=0.2, max_value=0.5, step=0.1)))

    # Conv Block 2
    model.add(layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=64, max_value=128, step=32),
        kernel_size=(3, 3),
        activation='relu',
        padding='same'
    ))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    

    # Klassificerings-delen
    model.add(layers.Flatten())
    model.add(layers.Dense(
        units=hp.Int('dense_units', min_value=128, max_value=512, step=128),
        activation='relu'
    ))
    model.add(layers.Dense(100, activation='softmax')) # 100 klasser
    

    # Vi kan även testa olika Learning Rates för optimeraren
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# 3. Starta Tunern

In [6]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    factor=3,
    directory='my_dir',
    project_name='cifar100_tuning',
    overwrite=True  
)

# En callback som stoppar om träningen inte blir bättre (sparar tid)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

C:\Users\Shaab\repos\AI DEL 2\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# 4. Kör sökningen!(Detta tar tid om du kör på all data)

In [7]:
tuner.search(x_train, y_train, epochs=10, validation_split=0.2, callbacks=[stop_early])

Trial 30 Complete [00h 00m 45s]
val_accuracy: 0.11699999868869781

Best val_accuracy So Far: 0.17100000381469727
Total elapsed time: 00h 17m 07s


# 5. Hämta den bästa modellen och dess hyperparametrar

In [8]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
Sökningen är klar! 
Optimalt antal filter i lager 1 var: {best_hps.get('conv_1_filter')}
Optimal learning rate var: {best_hps.get('learning_rate')}
""")


Sökningen är klar! 
Optimalt antal filter i lager 1 var: 96
Optimal learning rate var: 0.0001

